# `Member`: Retrospective

## What was the goal and intended behavior?

I wanted to mimic how `typing` annotations were used to try and document `Enum` members, potentially also giving them default values of their member names

For example, something like the following:
```python
class myEnum:
    SUNDAY:Member
    MONDAY:Member["Some documentation"]
    TUESDAY:Member["A value", "Some documentation"]
```

## Annotated source code for `classOrFunctionName`

In [1]:
import typing # We directly use the `typing` library to do this

In [29]:
# All `typing` classes inherit `typing._GenericAlias`
class _MemberGenericAlias(typing._GenericAlias, _root=True):
    # Creates a copy of the intended `typing` class for this `Alias`
    def copy_with(self, params):
        return Member[params]
    
    # Let `==` be utilized when comparing other members
    def __eq__(self, other):
        if not isinstance(other, _MemberGenericAlias):
            return NotImplemented
        return set(self.__args__) == set(other.__args__)
    
    # Let the class be picklable
    def __hash__(self):
        return hash(frozenset(self.__args__))
    
    # A clean repr, showing the `value` and the `doc`
    # Any arguments passed to MyType[a,b] (a,b in this case)
    # Are stored in `self.__args__`
    def __repr__(self): 
        args = list(self.__args__)
        if len(args) > 0:
            if isinstance(args[0], str):
                args[0] = f'"{args[0]}"'
            if len(args) == 2:
                # Value, Documentation
                if isinstance(args[1], str):
                    args[1] = f'"{args[1]}"'
                return f'Member[doc={args[0]}, value={args[1]}]'
            else:
                return f'Member[doc={args[0]}]'
        else:
            return f'Member[]'
    
    # For letting `isinstance()` work
    def __instancecheck__(self, obj):
        return self.__subclasscheck__(type(obj))
    
    # For letting `issubclass` work for each of the annotated types
    # Probably wasn't truly needed, since these are annotated documentation
    def __subclasscheck__(self, cls):
        for arg in self.__args__:
            if issubclass(cls, arg):
                return True
    
    # Tell pickle how to serialize
    def __reduce__(self):
        func, (origin, args) = super().__reduce__()
        return func, (Member, args)
    
    # Allow us to extract the passed value from the annotations
    # Value is stored as the second member in the annotation
    @property
    def value(self):
        args = self.__args__
        if len(self.__args__) == 2:
            return args[-1]
        return None
    
    # Allow us to extract the passed docstring from the annotation
    # The docstring is stored as the first member in the annotation
    @property
    def doc(self):
        args = self.__args__
        if len(self.__args__) > 0:
            return args[0]
        return None

In [30]:
# To be considered a "type" (and essentially not do anything/have special behaviors), we use the `typing.@_SpecialForm` decorator
@typing._SpecialForm
def Member(self, parameters):
    """Member type; Member[X, Y] means documentation of X, value of Y
    
    Used to quickly write documented Enums that may 
    have a value of the member name, in lower case.
    
    To define a member type:
      - Member["Some documentation"]
      - Member["Some documentation", "some_value"]
  
    If a Member has a single value, the Enum value will be the member name, in lower case,
    and the documentation will be that value
    
    If two, then the documentation the first, Enum value will be the second
    
    These attributes can be accessed with Member.value or Member.doc
    """
    if parameters == ():
        raise TypeError("Cannot take a Member without documentation")
    if not isinstance(parameters, tuple):
        parameters = (parameters,)
    return _MemberGenericAlias(self, parameters)

## Behaviors in action

In [31]:
from fastcore import test
# test_eq is a very basic assert test runner for ==

### Failed Version

In [33]:
t = Member["Some documentation"]
test.test_eq(t.doc, "Some documentation")
test.test_eq(t.value, None)
test.test_eq(t.__repr__(), 'Member[doc="Some documentation"]')

t2 = Member["Some documentation", "My value"]
test.test_eq(t2.doc, "Some documentation")
test.test_eq(t2.value, "My value")
test.test_eq(t2.__repr__(), 'Member[doc="Some documentation", value="My value"]')

t3 = Member["Some documentation", 3]
test.test_eq(t3.doc, "Some documentation")
test.test_eq(t3.value, 3)
test.test_eq(t3.__repr__(), 'Member[doc="Some documentation", value=3]')

### Version I went with

In [34]:
# All `typing` classes inherit `typing._GenericAlias`
class _MemberGenericAlias(typing._GenericAlias, _root=True):
    # Creates a copy of the intended `typing` class for this `Alias`
    def copy_with(self, params):
        return Member[params]
    
    # Let `==` be utilized when comparing other members
    def __eq__(self, other):
        if not isinstance(other, _MemberGenericAlias):
            return NotImplemented
        return set(self.__args__) == set(other.__args__)
    
    # Let the class be picklable
    def __hash__(self):
        return hash(frozenset(self.__args__))
    
    # A clean repr, showing the `doc`
    # Any arguments passed to MyType[a] (a in this case)
    # Are stored in `self.__args__`
    def __repr__(self): 
        args = list(self.__args__)
        if len(args) > 0:
            return f'Member[doc="{args[0]}"]'
        return f'Member[]'
    
    # For letting `isinstance()` work
    def __instancecheck__(self, obj):
        return self.__subclasscheck__(type(obj))
    
    # Tell pickle how to serialize
    def __reduce__(self):
        func, (origin, args) = super().__reduce__()
        return func, (Member, args)
    
    # Allow us to extract the passed docstring from the annotation
    @property
    def doc(self):
        args = self.__args__
        if len(self.__args__) > 0:
            return args[0]
        return None

In [35]:
# To be considered a "type" (and essentially not do anything/have special behaviors), we use the `typing.@_SpecialForm` decorator
@typing._SpecialForm
def Member(self, parameters):
    """Member type; Member[X] means documentation for the annotated item
    
    Used to quickly write documented Enums that may 
    have a value of the member name, in lower case.
    
    To define a member type:
      - Member["Some documentation"]
    
    If there is an assign statement after the Member typing, the value for that enum member will be it.
    Otherwise it will be the member name, in lower case:
    ```python
    someValue:Member # Will be "somevalue"
    someThing:Member["My thing"] # Will have documentaiton of "My thing" and a value of "something"
    someThing:Member["My thing"] = 2 # Will have documentation of "My thing" and a value of 2
    
    The documentation can be accessed with `Member.doc`
    """
    if parameters == ():
        raise TypeError("Cannot take a Member without documentation")
    if not isinstance(parameters, (tuple, list)):
        parameters = (parameters,)
    if len(parameters) > 1:
        raise ValueError(f'Member was passed more than a single value ({len(parameters)}). Please only pass in a docstring: {parameters}')
    return _MemberGenericAlias(self, parameters)

In [36]:
t = Member["Some documentation"]
test.test_eq(t.doc, "Some documentation")
test.test_eq(t.__repr__(), 'Member[doc="Some documentation"]')

with test.ExceptionExpected(ValueError, "Member was passed more than a single value"):
    t2 = Member["My value", "Some documentation"]

## Why this version did not make it in

Through looking at both the implementation and [twitter](https://twitter.com/TheZachMueller/status/1509352037640769536?s=20&t=SlhjMt_YDFP1XKqDsfhaXg) (but I will not deny the twitter influence), I realized that it was a bit *too* magical.

I originally let it not only take in a documentation, but also a default, leaving potential usage of it to look very non-pythonic:
```python
@enumify
class someEnum:
    MONDAY:Member["First day of the week", 0]
    TUESDAY:Member["Second day of the week"] # Will have a value of "tuesday"
    WEDNESDAY:Member # Will have a value of "wednesday" and no docstring
```

Since the goal with designing interfaces like this is it should be both intuitive to the user, and easy to read, I simplified it further to just have `Member` take in a docstring. This was based on feedback where my audience seemed to prefer an annotation like so:
```python
from typing import Any
@enumify
class someEnum:
    MONDAY:Any = "First day of the week"
    TUESDAY:Any = "Second day of the week", "Some value"
    WEDNESDAY:Any
```

To give us a final interface of:
```python
@enumify
class someEnum:
    MONDAY:Member["First day of the week"]
    TUESDAY:Member["Second day of the week"] = "Some value"
    WEDNESDAY:Member
    THURSDAY = 3
```

I am also considering allowing a shorthand of `Member` to be `Mem`, as I think the "`Any`" shorthand was also part of it's preference. This would leave an API with:

```python
@enumify
class someEnum:
    MONDAY:Mem["First day of the week"]
    TUESDAY:Mem["Second day of the week"] = "Some value"
    WEDNESDAY:Mem
    THURSDAY = 3
```

Another scope of influence I believe towards why many preferred the `Any` annotation is due to how Enum's are declared in Python. Typically it would be via an `=`, which leaves that behavior intact